# GP Regression on Application Project Data

---
Cell for importing packages:

In [2]:
# Install a pip package in the current Jupyter kernel
import sys
# !{sys.executable} -m pip install gpytorch

     |████████████████████████████████| 229 kB 685 kB/s eta 0:00:01
    Running setup.py install for gpytorch ... done


---
Imports cell:

In [6]:
# 441975, l.teixeira@wustl.edu, Teixeira, Lucas
# 443896, rickynoll@wustl.edu, Noll, Ricky
# XXXXXX, XXXXX@wustl.edu, Kowsari, Daria

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

---
## Read Training Input File:

In [7]:
# Read training input file
train = pd.read_csv(os.path.join(os.getcwd(),'train.csv'))
# Clip off labels from features and reset index
train_x = train.loc[:, 'ID':'Soil_Type'].set_index('ID')
# Extract labels into their own series
train_y = train.loc[:, ['ID','Horizontal_Distance_To_Fire_Points']].set_index('ID').squeeze()

# Do the same for the test file
test = pd.read_csv(os.path.join(os.getcwd(),'test.csv'))
test_x = test.set_index('ID')

X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, test_size=0.10, random_state=np.random.randint(1,654321))

print("X_train: ", X_train.shape)
print("X_val  : ", X_val.shape)
print("y_train: ", y_train.shape)
print("y_val  : ", y_val.shape)
print("test_x : ", test_x.shape)

X_train:  (6694, 10)
X_val  :  (744, 10)
y_train:  (6694,)
y_val  :  (744,)
test_x :  (11157, 10)


Now we have a training set, a validation set that we can compute metrics and stuff on, and our real test set for which we don't have labels but that we'll make our actual predictions on.

In [8]:
# See the first seven training examples
X_train[:7]

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Soil_Type
ID,,,,,,,,,,
7442324800189,3351,16,16,182,38,2132,204,205,134,8771
985993923065,3309,349,2,0,0,1250,215,235,158,8771
3572615222797,3169,345,3,30,0,638,214,235,159,7202
796578817163,3190,300,14,85,5,1195,179,233,194,8776
7185078174464,3406,100,20,633,211,1254,248,209,80,7755
8043930066694,3443,289,17,685,233,895,169,235,206,7757
8858564042712,3237,257,9,524,53,400,198,246,187,7202


In [9]:
# See labels of training samples
y_train.squeeze()

ID
7442324800189    2192
985993923065     1500
3572615222797    1179
796578817163     1168
7185078174464    1024
                 ... 
8945371371996    1648
506123677445     2018
5178812755885    3245
6797301770063     516
2068919655931    4308
Name: Horizontal_Distance_To_Fire_Points, Length: 6694, dtype: int64

---
## Let's Try GPyTorch:

In [10]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Construct our first, basic GP model:

First we need to turn our training data into `torch.Tensor`s

In [11]:
train_x_tensor = torch.tensor(X_train.to_numpy())
train_x_tensor

tensor([[3351,   16,   16,  ...,  205,  134, 8771],
        [3309,  349,    2,  ...,  235,  158, 8771],
        [3169,  345,    3,  ...,  235,  159, 7202],
        ...,
        [3443,   49,   10,  ...,  219,  128, 8703],
        [3319,  107,   11,  ...,  226,  116, 7756],
        [3320,  106,   23,  ...,  206,   70, 8776]])

In [12]:
train_y_tensor = torch.tensor(y_train.to_numpy())
train_y_tensor

tensor([2192, 1500, 1179,  ..., 3245,  516, 4308])

Now we define a boilerplate class for Exact GP Inference with standard constant mean function and RBF kernel.

In [13]:
# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
    
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x_tensor, train_y_tensor, likelihood)

With the model now defined we train by writing our own training loop.

In [14]:
# Run a thousand training iterations
training_iter = 1000

# Set model and likelihood to "train" (prior construction) mode
model.train()
likelihood.train()

hypers = {
    'likelihood.noise_covar.raw_noise': torch.tensor(44.2736),
    'mean_module.constant': torch.tensor(75.2567),
    'covar_module.raw_outputscale': torch.tensor(44.4823),
    'covar_module.base_kernel.raw_lengthscale': torch.tensor(103.6995),
}

model.initialize(**hypers)

# Use the adam optimizer
optimizer = torch.optim.Adam([
    {'params': model.parameters()},  # Includes GaussianLikelihood parameters
], lr=0.1)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for itr in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x_tensor)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y_tensor)
    loss.backward()
    if (itr + 1) % 50 == 0:
        print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
            itr + 1, training_iter, loss.item(),
            model.covar_module.base_kernel.lengthscale.item(),
            model.likelihood.noise.item()
        ))
    optimizer.step()

Iter 50/1000 - Loss: 3921.836   lengthscale: 108.377   noise: 49.070
Iter 100/1000 - Loss: 3533.376   lengthscale: 112.385   noise: 53.624
Iter 150/1000 - Loss: 3246.411   lengthscale: 115.696   noise: 57.880
Iter 200/1000 - Loss: 3022.206   lengthscale: 118.461   noise: 61.901
Iter 250/1000 - Loss: 2839.062   lengthscale: 120.789   noise: 65.730
Iter 300/1000 - Loss: 2687.003   lengthscale: 122.754   noise: 69.397
Iter 350/1000 - Loss: 2556.977   lengthscale: 124.423   noise: 72.923
Iter 400/1000 - Loss: 2445.304   lengthscale: 125.840   noise: 76.327
Iter 450/1000 - Loss: 2345.929   lengthscale: 127.057   noise: 79.622
Iter 500/1000 - Loss: 2258.313   lengthscale: 128.095   noise: 82.821
Iter 550/1000 - Loss: 2179.409   lengthscale: 128.985   noise: 85.933
Iter 600/1000 - Loss: 2107.438   lengthscale: 129.750   noise: 88.968
Iter 650/1000 - Loss: 2041.828   lengthscale: 130.384   noise: 91.930
Iter 700/1000 - Loss: 1981.439   lengthscale: 130.918   noise: 94.824
Iter 750/1000 - Loss:

Now that we have a trained model let's take a look at our hyperparameter values

In [15]:
for param_name, param in model.named_parameters():
    print(f'Parameter name: {param_name:42} value = {param.item()}')

Parameter name: likelihood.noise_covar.raw_noise           value = 111.09373474121094
Parameter name: mean_module.constant                       value = 144.70777893066406
Parameter name: covar_module.raw_outputscale               value = 102.5657730102539
Parameter name: covar_module.base_kernel.raw_lengthscale   value = 132.8197021484375


Now we'll use our validation set to compute regression metrics to see how well we did.

First we make our validation points tensor:

In [16]:
val_x_tensor = torch.tensor(X_val.to_numpy())
val_x_tensor

tensor([[3185,  358,    5,  ...,  230,  156, 7700],
        [3252,  238,    9,  ...,  248,  182, 7202],
        [3132,  148,   16,  ...,  236,  120, 7756],
        ...,
        [3147,  149,   22,  ...,  233,  108, 7756],
        [3306,  340,   21,  ...,  205,  170, 7202],
        [3409,   62,   15,  ...,  208,  106, 8776]])

Now we set our model and likelihood to prediction mode and do constant time predictions with `gpytorch.settings.fast_pred_var()`

In [17]:
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

# Make validation predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    validation_preds = likelihood(model(val_x_tensor))

The likelihood returns us a MultivariateNormal object that represents the posterior predictive distribution of the validation points.

In [18]:
validation_preds

MultivariateNormal(loc: torch.Size([744]))

We use the mean of this distribution as our predictions.

In [19]:
validation_preds.mean

tensor([ 717.5021, 1789.9164, 2585.5229, 3599.7715, 4656.9980, 1305.0979,
        1028.6925, 3062.2036, 1164.3169, 2996.1514, 2889.7136,  877.8260,
        3163.8708, 2972.9346,  642.3585, 3085.5698, 1005.1996,  613.7988,
        3411.9556, 1439.0464, 1502.6738, 1618.6664,  310.5034, 1512.4285,
        2296.1899, 1792.6855, 1144.6295, 2098.0798, 1690.4912, 1062.9127,
        1130.6365, 4143.9692, 2149.9348,  813.8082, 1910.8828, 3624.2971,
        1610.8384,  685.6976, 1150.2189, 1658.9412, 1282.7362,  910.6946,
        1078.7549, 3121.1594, 1824.6714, 2572.5872, 2493.7512,  855.3162,
        2440.5925, 1264.7688, 1191.6970, 1587.5911, 5006.6060, 1234.1038,
        2290.5728, 1228.5254,  712.1539, 1411.8784, 1884.2002, 1809.8217,
        2094.0547,  723.2311,  934.2905,  364.3827, 1362.7466, 1703.1594,
        2333.0759, 2809.0322,  800.8282, 3428.0227, 1096.4874, 1543.6919,
        3616.0393, 4160.1084, 2704.2129,  703.2686,  906.6664, 1567.8239,
        4823.0703, 2027.8823, 2784.664

So we use the mean to plug into our regression metrics to see how we did

In [20]:
# Use the posterior mean as our prediction
print("--- Regression Metrics ---")
mse = mean_squared_error(y_val, validation_preds.mean)
mae = mean_absolute_error(y_val, validation_preds.mean)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, validation_preds.mean)

print(f"\nMSE : {mse}")
print(f"\nRMSE: {rmse}")
print(f"\nMAE : {mae}")
print(f"\nR^2  : {r2}")

--- Regression Metrics ---

MSE : 190343.28697847307

RMSE: 436.28349381849534

MAE : 285.5033655474263

R^2  : 0.8991097737010715


---
### Make real predictions and pipe output to file

In [32]:
test_x_tensor = torch.tensor(test_x.to_numpy())
test_x_tensor

tensor([[3229,   98,   22,  ...,  204,   72, 7757],
        [3324,   92,   18,  ...,  209,   86, 8776],
        [3433,  162,   11,  ...,  243,  143, 8771],
        ...,
        [3446,  274,    4,  ...,  240,  169, 8703],
        [3372,  281,   11,  ...,  241,  192, 7755],
        [3132,  129,    9,  ...,  235,  130, 7756]])

In [34]:
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    test_preds = likelihood(model(test_x_tensor))
    submission_output = pd.DataFrame(data={'ID': test_x.index, 'Horizontal_Distance_To_Fire_Points': test_preds.mean})
    submission_output.to_csv(os.path.join(os.getcwd(),'simple_gp_predictions.csv'), index=False)
submission_output